# Histogram Mode Detector

## IO

I/O providers can load white beam images and slice them according to the image key.

Note that not like the ``rotation_angle``, ``image_key`` is not assigned as a coordinate.

Instead, we slice the data according to the ``image_key`` and separate image stacks earlier due to performance limitation of 3-dimensional data binning.

> For example, for 400 images of 2_048 x 2_048 pixels couldn't be done in the regular laptop.

In [ ]:
from ess.imaging.data import get_ymir_images_path
from ess.imaging.io import (
    FilePath,
    SampleImageStacks,
    RawSampleImageStacks,
    OpenBeamImageStacks,
    DarkCurrentImageStacks,
    AllImageStacks,
)
from ess.imaging.workflow import (
    YmirWorkflow,
    ImageDetectorName,
    RotationMotionSensorName,
)

wf = YmirWorkflow()
wf[FilePath] = get_ymir_images_path()
wf[ImageDetectorName] = 'orca'
wf[RotationMotionSensorName] = 'motion_cabinet_2'

In [ ]:
image_stack_types = (
    AllImageStacks,
    RawSampleImageStacks,
    OpenBeamImageStacks,
    DarkCurrentImageStacks,
    SampleImageStacks,
)
wf.visualize(image_stack_types)

In [ ]:
results = wf.compute(image_stack_types)
results[SampleImageStacks]

## Save Individual Frame

In [ ]:
from ess.imaging.io import export_image_stacks_as_tiff, ImageKey

from pathlib import Path
from tqdm import tqdm


output_dir = Path("images")
export_image_stacks_as_tiff(
    output_dir=output_dir,
    image_stacks=results[AllImageStacks],
    merge_image_by_key=False,
    overwrite=True,
    progress_wrapper=tqdm,
    image_prefix_map={
        ImageKey.SAMPLE: "sample",
        ImageKey.DARK_CURRENT: "dc",
        ImageKey.OPEN_BEAM: "ob",
    },
)

In [ ]:
sorted(output_dir.iterdir())

## Save All Frames in One File

In [ ]:
from ess.imaging.io import export_image_stacks_as_tiff, ImageKey
from tqdm import tqdm
from pathlib import Path


output_dir = Path("images")
export_image_stacks_as_tiff(
    output_dir=output_dir,
    image_stacks=results[AllImageStacks],
    merge_image_by_key=True,
    overwrite=True,
    progress_wrapper=tqdm,
    image_prefix_map={
        ImageKey.SAMPLE: "sample",
        ImageKey.DARK_CURRENT: "dc",
        ImageKey.OPEN_BEAM: "ob",
    },
)

In [ ]:
tuple(output_dir.iterdir())

In [ ]:
!rm -rf images